In [1]:
import os
import PIL.Image as Image
import torch.cuda
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import torchvision
from LookGenerator.networks.clothes_feature_extractor import ClothingAutoEncoder
from LookGenerator.datasets.basic_dataset import BasicDataset
from LookGenerator.networks.losses import VAELoss
from LookGenerator.networks.utils import load_model
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from torchsummary import summary
from LookGenerator.networks.utils import get_num_digits, save_model
from LookGenerator.networks_training.utils import check_path_and_creat

KeyboardInterrupt: 

In [2]:
transform_input = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [3]:
batch_size_train = 32
batch_size_val = 16
pin_memory = True
num_workers = 4

In [4]:
train_dataset = BasicDataset(
    root_dir=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    dir_name=r"cloth",
    transform_input=transform_input
)
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size_train, shuffle=True, pin_memory=pin_memory,
    num_workers=num_workers
)

In [5]:
val_dataset = BasicDataset(
    root_dir=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\test",
    dir_name="cloth",
    transform_input=transform_input
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size_train, shuffle=False, pin_memory=pin_memory,
    num_workers=num_workers
)

In [6]:
print(len(val_dataset), len(val_dataloader))
print(len(train_dataset), len(train_dataloader))

2032 64
11647 364


In [7]:
def _epoch_string(epoch, epoch_num):
    num_digits_epoch_num = get_num_digits(epoch_num)
    num_digits_epoch = get_num_digits(epoch)

    epoch_string = "0"*(num_digits_epoch_num - num_digits_epoch) + str(epoch)
    return epoch_string

In [8]:
def fit(model, criterion, optimizer, device, train_loader, val_loader, epochs, save_directory, save_step=1):

    train_loss=[]
    val_loss=[]

    criterion = criterion.to(device)
    model = model.to(device)

    for epoch in range(epochs):
        torch.cuda.empty_cache()
        model.to(device)
        model.train()

        train_epoch_loss = []
        for X_batch in tqdm(train_loader):
            X_batch = X_batch.to(device)
            optimizer.zero_grad()
            pred = model(X_batch)
            #reconstructed, mu, log_var = pred
            #loss = criterion(X_batch, mu, log_var, reconstructed)
            loss = criterion(pred, X_batch)
            loss.backward()
            optimizer.step()
            # print(mu, log_var)
            train_epoch_loss.append(loss.item())
            # print(loss.item())
        train_loss.append(np.mean(train_epoch_loss))

        model.eval()
        val_loss_epoch = []
        with torch.no_grad():
          for X_val in tqdm(val_loader):
              X_val = X_val.to(device)
              pred = model(X_val)
              #reconstructed, mu, log_var  = pred
              loss = criterion(reconstructed, X_val)
              val_loss_epoch.append(loss.item())
        val_loss.append(np.mean(val_loss_epoch))

        print("Epoch [{}/{}], train_loss: {:.3f}, val_loss: {:.3f}".format(
            epoch+1, epochs,
            train_loss[-1], val_loss[-1])
        )

        if (epoch + 1) % save_step == 0:
            save_model(
            model.to('cpu'),
            path=f"{save_directory}\\epoch_{_epoch_string(epoch, epochs)}.pt"
        )

    return train_loss, val_loss

In [9]:
save_directory=r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\autoDegradation\weights\session1"
check_path_and_creat(save_directory)

True

In [10]:
criterion = nn.MSELoss() # VAELoss()
model = ClothingAutoEncoder(latent_dim=200)
optimizer = torch.optim.Adam(model.parameters(), lr = 10e-4)
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_loss, val_loss = fit(model=model,
                           criterion=criterion,
                           optimizer=optimizer,
                           train_loader=train_dataloader,
                           val_loader=val_dataloader,
                           epochs=30,
                           save_directory=save_directory,
                           device=device)

100%|██████████| 64/64 [00:12<00:00,  5.27it/s]


Epoch [1/30], train_loss: 2559726.625, val_loss: 20698358.626


100%|██████████| 64/64 [00:11<00:00,  5.39it/s]


Epoch [2/30], train_loss: 7117009585552.180, val_loss: 424159227239881.625


100%|██████████| 64/64 [00:11<00:00,  5.38it/s]


...

100%|██████████| 64/64 [00:11<00:00,  5.70it/s]


Epoch [6/30], train_loss: 158839.281, val_loss: 3413.255


100%|██████████| 64/64 [00:13<00:00,  4.88it/s]


Epoch [7/30], train_loss: 96460069.509, val_loss: 80477.371


 10%|▉         | 36/364 [00:13<00:59,  5.48it/s]

In [12]:
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.legend()
plt.show()

NameError: name 'train_loss' is not defined

In [13]:
model.eval()
model.to('cpu')
for X_val in val_dataloader:
  reconstructed, mu, log_var = model(X_val)
  img = transforms.ToPILImage()(reconstructed[0]/2+0.5)
  cl = transforms.ToPILImage()(X_val[0])
  cl.show()
  img.show()
  break